# Wikipedia training

### In this tutorial we will:
 - Learn how to train the NMF topic model on the English Wikipedia corpus
 - Compare it with LDA and Sklearn NMF
 - Evaluate results

In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import json
import logging
import numpy as np
import pandas as pd
import scipy.sparse
from smart_open import smart_open
import time
import os
import psutil
from contextlib import contextmanager
from multiprocessing import Process
from tqdm import tqdm, tqdm_notebook
import joblib

import gensim.downloader as api
from gensim import matutils
from gensim.corpora import MmCorpus, Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.nmf import Nmf as GensimNmf
from sklearn.decomposition.nmf import NMF as SklearnNmf
from gensim.parsing.preprocessing import preprocess_string

tqdm.pandas()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.INFO)

# Preprocessing

### Load wikipedia dump
Let's use `gensim.downloader.api` for that

In [2]:
data = api.load("wiki-english-20171001")
article = next(iter(data))

for section_title, section_text in zip(
    article['section_titles'],
    article['section_texts']
):
    print("Section title: %s" % section_title)
    print("Section text: %s" % section_text[:100])

Section title: Introduction
Section text: 




'''Anarchism''' is a political philosophy that advocates self-governed societies based on volun
Section title: Etymology and terminology
Section text: 

The word ''anarchism'' is composed from the word ''anarchy'' and the suffix ''-ism'', themselves d
Section title: History
Section text: 

===Origins===
Woodcut from a Diggers document by William Everard

The earliest anarchist themes ca
Section title: Anarchist schools of thought
Section text: 
Portrait of philosopher Pierre-Joseph Proudhon (1809–1865) by Gustave Courbet. Proudhon was the pri
Section title: Internal issues and debates
Section text: 
consistent with anarchist values is a controversial subject among anarchists.

Anarchism is a philo
Section title: Topics of interest
Section text: Intersecting and overlapping between various schools of thought, certain topics of interest and inte
Section title: Criticisms
Section text: 
Criticisms of anarchism include moral criticisms and pra

Preprocess and save articles

In [3]:
def save_preprocessed_articles(filename, articles):
    with smart_open(filename, 'w+', encoding="utf8") as writer:
        for article in tqdm_notebook(articles):
            article_text = " ".join(
                " ".join(section)
                for section
                in zip(
                    article['section_titles'],
                    article['section_texts']
                )
            )
            article_text = preprocess_string(article_text)

            writer.write(json.dumps(article_text) + '\n')


def get_preprocessed_articles(filename):
    with smart_open(filename, 'r', encoding="utf8") as reader:
        for line in tqdm_notebook(reader):
            yield json.loads(
                line
            )

In [4]:
SAVE_ARTICLES = False

if SAVE_ARTICLES:
    save_preprocessed_articles('wiki_articles.jsonlines', data)

### Create and save dictionary

In [5]:
SAVE_DICTIONARY = False

if SAVE_DICTIONARY:
    dictionary = Dictionary(get_preprocessed_articles('wiki_articles.jsonlines'))
    dictionary.save('wiki.dict')

### Load and filter dictionary

In [6]:
dictionary = Dictionary.load('wiki.dict')
dictionary.filter_extremes()
dictionary.compactify()

2019-02-04 13:43:35,082 : INFO : loading Dictionary object from wiki.dict
2019-02-04 13:43:37,789 : INFO : loaded wiki.dict
2019-02-04 13:43:50,640 : INFO : discarding 1910146 tokens: [('abdelrahim', 49), ('abstention', 120), ('anarcha', 101), ('anarchica', 40), ('anarchosyndicalist', 20), ('antimilitar', 68), ('arbet', 194), ('archo', 100), ('arkhē', 5), ('autonomedia', 118)]...
2019-02-04 13:43:50,642 : INFO : keeping 100000 tokens which were in no less than 5 and no more than 2462447 (=50.0%) documents
2019-02-04 13:43:52,106 : INFO : resulting dictionary: Dictionary(100000 unique tokens: ['miwa', 'imagin', 'streamsid', 'slavist', 'magyarország']...)


### MmCorpus wrapper
In this way we'll:

- Make sure that documents are shuffled
- Be able to train-test split corpus without rewriting it

In [7]:
class RandomCorpus(MmCorpus):
    def __init__(self, random_seed=42, testset=False, testsize=1000, *args,
                 **kwargs):
        super().__init__(*args, **kwargs)

        random_state = np.random.RandomState(random_seed)
        
        self.indices = random_state.permutation(range(self.num_docs))
        test_nnz = sum(len(self[doc_idx]) for doc_idx in self.indices[:testsize])
        
        if testset:
            self.indices = self.indices[:testsize]
            self.num_docs = testsize
            self.num_nnz = test_nnz
        else:
            self.indices = self.indices[testsize:]
            self.num_docs -= testsize
            self.num_nnz -= test_nnz

    def __iter__(self):
        for doc_id in self.indices:
            yield self[doc_id]

### Create and save corpus

In [8]:
SAVE_CORPUS = False

if SAVE_CORPUS:
    corpus = (
        dictionary.doc2bow(article)
        for article
        in get_preprocessed_articles('wiki_articles.jsonlines')
    )
    
    RandomCorpus.serialize('wiki.mm', corpus)

### Load train and test corpus
Using `RandomCorpus` wrapper

In [9]:
train_corpus = RandomCorpus(
    random_seed=42, testset=False, testsize=2000, fname='wiki.mm'
)
test_corpus = RandomCorpus(
    random_seed=42, testset=True, testsize=2000, fname='wiki.mm'
)

2019-02-04 13:43:53,035 : INFO : loaded corpus index from wiki.mm.index
2019-02-04 13:43:53,036 : INFO : initializing cython corpus reader from wiki.mm
2019-02-04 13:43:53,037 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries
2019-02-04 13:43:55,051 : INFO : loaded corpus index from wiki.mm.index
2019-02-04 13:43:55,052 : INFO : initializing cython corpus reader from wiki.mm
2019-02-04 13:43:55,053 : INFO : accepted corpus with 4924894 documents, 100000 features, 683326444 non-zero entries


### Convert corpora to csc and save

It's necessary in order to train Sklearn NMF. These matrices take **a lot** of RAM even though they're sparse (about 8GB).

In [10]:
SAVE_CSC = False

if SAVE_CSC:
    train_csc = matutils.corpus2csc(train_corpus, len(dictionary))
    scipy.sparse.save_npz('train_csc.npz', train_csc)
    
    test_csc = matutils.corpus2csc(test_corpus, len(dictionary))
    scipy.sparse.save_npz('test_csc.npz', test_csc)

## Metrics

- `train time` - time to train a model
- `mean_ram` - mean RAM consumption during the training
- `max_ram` - maximum RAM consumpiton during the training
- `perplexity` - perplexity score. Another usual TM metric
- `coherence` - coherence score (not defined for sklearn NMF). Classic metric for topic models.
- `l2_norm` - l2 norm of `v - Wh`

In [11]:
@contextmanager
def measure_ram(output, tick=2):
    def _measure_ram(pid, output, start_memory, tick=5):
        py = psutil.Process(pid)
        with open(output, 'w') as outfile:
            while True:
                memory = py.memory_info().rss - start_memory
                outfile.write("{}\n".format(memory))
                outfile.flush()
                time.sleep(tick)

    pid = os.getpid()
    start_memory = psutil.Process(pid).memory_info().rss
    p = Process(target=_measure_ram, args=(pid, output, start_memory, 5))
    p.start()
    yield
    p.terminate()

def get_train_time_and_ram(func, name):
    memprof_filename = "{}.memprof".format(name)
    
    start = time.time()

    with measure_ram(memprof_filename, 5):
        result = func()        
        
    elapsed_time = pd.to_timedelta(time.time() - start, unit='s').round('s')
    
    memprof_df = pd.read_csv(memprof_filename, squeeze=True)
    
    mean_ram = "{} MB".format(
        int(memprof_df.mean() // 2**20),
    )
    
    max_ram = "{} MB".format(int(memprof_df.max() // 2**20))

    return elapsed_time, mean_ram, max_ram, result


def get_tm_metrics(model, test_corpus):
    W = model.get_topics().T
    H = np.zeros((model.num_topics, len(test_corpus)))
    for bow_id, bow in enumerate(test_corpus):
        for topic_id, word_count in model.get_document_topics(bow):
            H[topic_id, bow_id] = word_count

    pred_factors = W.dot(H)
    
    dense_corpus = matutils.corpus2dense(test_corpus, pred_factors.shape[0])

    l2_norm = get_tm_l2_norm(pred_factors, dense_corpus)
    
    pred_factors /= pred_factors.sum(axis=0)
    
    perplexity = get_tm_perplexity(pred_factors, dense_corpus)

    model.normalize = True

    coherence = CoherenceModel(
        model=model,
        corpus=test_corpus,
        coherence='u_mass'
    ).get_coherence()
    
    topics = model.show_topics(5)

    model.normalize = False

    return dict(
        perplexity=round(perplexity, 4),
        coherence=round(coherence, 4),
        l2_norm=round(l2_norm, 4),
        topics=topics,
    )


def get_tm_perplexity(pred_factors, dense_corpus):
    return np.exp(-(np.log(pred_factors, where=pred_factors > 0) * dense_corpus).sum() / dense_corpus.sum())


def get_tm_l2_norm(pred_factors, dense_corpus):
    return np.linalg.norm(dense_corpus - pred_factors)


def get_sklearn_topics(model, id2word, top_n=5):
    topic_probas = model.components_.T
    topic_probas = topic_probas / topic_probas.sum(axis=0)
    
    sparsity = np.zeros(topic_probas.shape[1])

    for row in topic_probas:
        sparsity += (row == 0)

    sparsity /= topic_probas.shape[1]
    
    topic_probas = topic_probas[:, sparsity.argsort()[::-1]][:, :top_n]
    
    token_indices = topic_probas.argsort(axis=0)[:-11:-1, :]
    topic_probas.sort(axis=0)
    topic_probas = topic_probas[:-11:-1, :]
    
    topics = []
    
    for topic_idx in range(topic_probas.shape[1]):
        tokens = [
            id2word[token_idx]
            for token_idx
            in token_indices[:, topic_idx]
        ]
        topic = (
            '{}*"{}"'.format(round(proba, 3), token)
            for proba, token
            in zip(topic_probas[:, topic_idx], tokens)
        )
        topic = " + ".join(topic)
        topics.append((topic_idx, topic))
    
    return topics


def get_sklearn_metrics(model, test_corpus, dictionary):
    W = model.components_.T
    H = model.transform((test_corpus).T).T
    pred_factors = W.dot(H)
    
    l2_norm = np.linalg.norm(test_corpus - pred_factors)
    
    pred_factors /= pred_factors.sum(axis=0)

    perplexity = np.exp(
        -(np.log(pred_factors, where=pred_factors > 0) * test_corpus).sum()
        / test_corpus.sum()
    )
    
    topics = get_sklearn_topics(model, dictionary, top_n=5)

    return dict(
        perplexity=perplexity,
        l2_norm=l2_norm,
        topics=topics,
    )

Define the dataframe in which we'll store metrics

In [12]:
tm_metrics = pd.DataFrame(columns=[
    'model', 'train_time', 'mean_ram', 'max_ram', 'perplexity', 'coherence', 'l2_norm', 'topics'
])

### Define common params for the models

In [13]:
params = dict(
    corpus=train_corpus,
    chunksize=2000,
    num_topics=50,
    id2word=dictionary,
    passes=1,
    eval_every=10,
    minimum_probability=0,
    random_state=42,
)

## Training

### Train Gensim NMF and save it
Normalization is turned off to compute metrics correctly

In [14]:
row = dict()
row['model'] = 'gensim_nmf'
row['train_time'], row['mean_ram'], row['max_ram'], nmf = get_train_time_and_ram(
    lambda: GensimNmf(
        normalize=False,
        **params
    ),
    'gensim_nmf',
)

nmf.save('gensim_nmf.model')

2019-02-04 13:44:40,694 : INFO : Loss: 0.04250465036259249
2019-02-04 13:45:11,600 : INFO : Loss: 0.035834409283392266
2019-02-04 13:45:26,724 : INFO : Loss: 0.01666296904388558
2019-02-04 13:45:39,544 : INFO : Loss: 0.014592172927081915
2019-02-04 13:45:58,732 : INFO : Loss: 0.03738619426262098
2019-02-04 13:46:15,154 : INFO : Loss: 0.011548734523320468
2019-02-04 13:46:28,638 : INFO : Loss: 0.005949484242001706
2019-02-04 13:46:39,398 : INFO : Loss: 0.0054164438291086905
2019-02-04 13:46:50,311 : INFO : Loss: 0.005098339670066033
2019-02-04 13:47:00,470 : INFO : Loss: 0.0042822453766947766
2019-02-04 13:47:13,109 : INFO : Loss: 0.004218452750957835
2019-02-04 13:47:27,197 : INFO : Loss: 0.004957368001488409
2019-02-04 13:47:36,715 : INFO : Loss: 0.004023650353364507
2019-02-04 13:47:47,268 : INFO : Loss: 0.004060624865162843
2019-02-04 13:47:57,039 : INFO : Loss: 0.0007936539858723043
2019-02-04 13:48:05,893 : INFO : Loss: 0.002926750080973836
2019-02-04 13:48:14,663 : INFO : Loss: 0

2019-02-04 14:00:49,084 : INFO : Loss: 0.00028238131271271424
2019-02-04 14:00:54,438 : INFO : Loss: 0.0002640478005720415
2019-02-04 14:00:59,685 : INFO : Loss: 0.00022538729252012077
2019-02-04 14:01:04,615 : INFO : Loss: 0.0
2019-02-04 14:01:09,911 : INFO : Loss: 0.0
2019-02-04 14:01:15,330 : INFO : Loss: 0.00024749787367713824
2019-02-04 14:01:20,718 : INFO : Loss: 0.0005803275246304228
2019-02-04 14:01:26,062 : INFO : Loss: 0.00016246434292861256
2019-02-04 14:01:30,978 : INFO : Loss: 0.0003649175874610073
2019-02-04 14:01:36,354 : INFO : Loss: 0.00043936818925609906
2019-02-04 14:01:41,656 : INFO : Loss: 0.0
2019-02-04 14:01:46,967 : INFO : Loss: 0.0001473140708686702
2019-02-04 14:01:52,093 : INFO : Loss: 0.00030760951796811754
2019-02-04 14:01:57,439 : INFO : Loss: 0.0001534249170939211
2019-02-04 14:02:02,862 : INFO : Loss: 0.0002268004146246076
2019-02-04 14:02:08,061 : INFO : Loss: 0.00020589154304862873
2019-02-04 14:02:13,190 : INFO : Loss: 0.00042309497830752803
2019-02-0

### Load Gensim NMF and store metrics

In [15]:
nmf = GensimNmf.load('gensim_nmf.model')
row.update(get_tm_metrics(nmf, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-02-04 14:10:06,115 : INFO : loading Nmf object from gensim_nmf.model
2019-02-04 14:10:06,425 : INFO : loading id2word recursively from gensim_nmf.model.id2word.* with mmap=None
2019-02-04 14:10:06,425 : INFO : loaded gensim_nmf.model
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-02-04 14:10:46,336 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2019-02-04 14:10:46,447 : INFO : CorpusAccumulator accumulated stats from 2000 documents


### Train LDA and save it
That's a common model to do Topic Modeling

In [16]:
row = dict()
row['model'] = 'lda'
row['train_time'], row['mean_ram'], row['max_ram'], lda = get_train_time_and_ram(
    lambda: LdaModel(**params),
    'lda',
)
lda.save('lda.model')

2019-02-04 14:10:46,611 : INFO : using symmetric alpha at 0.02
2019-02-04 14:10:46,613 : INFO : using symmetric eta at 0.02
2019-02-04 14:10:46,630 : INFO : using serial LDA version on this node
2019-02-04 14:10:47,152 : INFO : running online (single-pass) LDA training, 50 topics, 1 passes over the supplied corpus of 4922894 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2019-02-04 14:10:47,309 : INFO : PROGRESS: pass 0, at document #2000/4922894
2019-02-04 14:10:49,255 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-04 14:10:49,573 : INFO : topic #36 (0.020): 0.006*"new" + 0.005*"record" + 0.004*"servic" + 0.004*"includ" + 0.003*"year" + 0.003*"state" + 0.003*"design" + 0.003*"work" + 0.003*"time" + 0.003*"american"
2019-02-04 14:10:49,575 : INFO : topic #19 (0.020): 0.005*"team" + 0.005*"new" + 0.004*"nation" + 0.004*"state" + 0.004*"includ"

2019-02-04 14:11:00,784 : INFO : topic diff=0.220294, rho=0.408248
2019-02-04 14:11:00,943 : INFO : PROGRESS: pass 0, at document #14000/4922894
2019-02-04 14:11:02,595 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-04 14:11:02,855 : INFO : topic #39 (0.020): 0.038*"team" + 0.025*"season" + 0.025*"coach" + 0.020*"leagu" + 0.015*"plai" + 0.012*"player" + 0.012*"footbal" + 0.010*"hockei" + 0.009*"year" + 0.009*"basketbal"
2019-02-04 14:11:02,856 : INFO : topic #0 (0.020): 0.035*"station" + 0.031*"art" + 0.015*"line" + 0.014*"railwai" + 0.012*"museum" + 0.012*"paint" + 0.012*"left" + 0.011*"align" + 0.010*"work" + 0.009*"artist"
2019-02-04 14:11:02,857 : INFO : topic #13 (0.020): 0.020*"south" + 0.018*"north" + 0.015*"west" + 0.013*"east" + 0.012*"castl" + 0.012*"lake" + 0.010*"counti" + 0.009*"villag" + 0.008*"vallei" + 0.008*"hospit"
2019-02-04 14:11:02,858 : INFO : topic #45 (0.020): 0.015*"design" + 0.007*"snake" + 0.006*"standard" + 0.006*"depi

2019-02-04 14:11:17,839 : INFO : topic #10 (0.020): 0.010*"cell" + 0.009*"diseas" + 0.007*"cancer" + 0.007*"gene" + 0.007*"acid" + 0.007*"caus" + 0.006*"patient" + 0.006*"treatment" + 0.006*"protein" + 0.005*"includ"
2019-02-04 14:11:17,841 : INFO : topic #18 (0.020): 0.007*"man" + 0.006*"kill" + 0.005*"charact" + 0.005*"time" + 0.004*"seri" + 0.004*"later" + 0.004*"appear" + 0.004*"episod" + 0.004*"stori" + 0.004*"father"
2019-02-04 14:11:17,842 : INFO : topic #0 (0.020): 0.055*"station" + 0.038*"art" + 0.024*"line" + 0.024*"railwai" + 0.016*"paint" + 0.016*"work" + 0.013*"align" + 0.012*"train" + 0.012*"museum" + 0.012*"london"
2019-02-04 14:11:17,843 : INFO : topic #37 (0.020): 0.007*"love" + 0.007*"time" + 0.006*"plai" + 0.006*"releas" + 0.005*"appear" + 0.005*"night" + 0.004*"man" + 0.004*"song" + 0.003*"girl" + 0.003*"year"
2019-02-04 14:11:17,844 : INFO : topic #35 (0.020): 0.022*"russian" + 0.020*"russia" + 0.018*"parti" + 0.017*"philippin" + 0.013*"union" + 0.013*"moscow" + 0.

2019-02-04 14:11:29,841 : INFO : topic #25 (0.020): 0.016*"mountain" + 0.015*"river" + 0.015*"park" + 0.014*"lake" + 0.013*"mount" + 0.011*"area" + 0.008*"peak" + 0.007*"rock" + 0.006*"natur" + 0.006*"survei"
2019-02-04 14:11:29,842 : INFO : topic #27 (0.020): 0.045*"race" + 0.015*"dai" + 0.012*"team" + 0.012*"tour" + 0.012*"time" + 0.010*"rider" + 0.010*"hors" + 0.010*"stage" + 0.010*"finish" + 0.009*"second"
2019-02-04 14:11:29,844 : INFO : topic #48 (0.020): 0.066*"octob" + 0.062*"januari" + 0.061*"march" + 0.057*"septemb" + 0.055*"novemb" + 0.053*"june" + 0.053*"april" + 0.052*"juli" + 0.051*"august" + 0.051*"decemb"
2019-02-04 14:11:29,850 : INFO : topic diff=0.286263, rho=0.229416
2019-02-04 14:11:33,807 : INFO : -11.672 per-word bound, 3262.6 perplexity estimate based on a held-out corpus of 2000 documents with 564313 words
2019-02-04 14:11:33,807 : INFO : PROGRESS: pass 0, at document #40000/4922894
2019-02-04 14:11:35,383 : INFO : merging changes from 2000 documents into a mod

2019-02-04 14:11:45,627 : INFO : topic #21 (0.020): 0.025*"san" + 0.021*"spanish" + 0.019*"mexico" + 0.016*"del" + 0.012*"juan" + 0.012*"spain" + 0.012*"mexican" + 0.011*"puerto" + 0.010*"santa" + 0.010*"josé"
2019-02-04 14:11:45,628 : INFO : topic #49 (0.020): 0.028*"india" + 0.022*"indian" + 0.007*"singh" + 0.006*"peopl" + 0.006*"pakistan" + 0.006*"tropic" + 0.006*"ali" + 0.006*"pradesh" + 0.006*"area" + 0.006*"khan"
2019-02-04 14:11:45,634 : INFO : topic diff=0.237375, rho=0.200000
2019-02-04 14:11:45,787 : INFO : PROGRESS: pass 0, at document #52000/4922894
2019-02-04 14:11:47,353 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-04 14:11:47,618 : INFO : topic #21 (0.020): 0.027*"san" + 0.020*"spanish" + 0.019*"mexico" + 0.018*"del" + 0.013*"juan" + 0.011*"mexican" + 0.011*"spain" + 0.011*"josé" + 0.010*"puerto" + 0.010*"rico"
2019-02-04 14:11:47,619 : INFO : topic #25 (0.020): 0.022*"river" + 0.015*"lake" + 0.015*"mountain" + 0.012*"area" + 0.0

...

2019-02-04 15:37:37,101 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-04 15:37:37,367 : INFO : topic #22 (0.020): 0.034*"speci" + 0.017*"famili" + 0.012*"plant" + 0.010*"genu" + 0.010*"white" + 0.008*"bird" + 0.008*"median" + 0.007*"tree" + 0.006*"incom" + 0.006*"flower"
2019-02-04 15:37:37,369 : INFO : topic #30 (0.020): 0.035*"leagu" + 0.035*"club" + 0.032*"plai" + 0.027*"team" + 0.024*"season" + 0.023*"cup" + 0.021*"footbal" + 0.017*"goal" + 0.015*"match" + 0.012*"score"
2019-02-04 15:37:37,370 : INFO : topic #5 (0.020): 0.038*"album" + 0.028*"song" + 0.027*"releas" + 0.025*"record" + 0.021*"band" + 0.016*"music" + 0.016*"singl" + 0.014*"chart" + 0.013*"track" + 0.010*"northshor"
2019-02-04 15:37:37,371 : INFO : topic #3 (0.020): 0.032*"presid" + 0.025*"nation" + 0.024*"offic" + 0.024*"govern" + 0.023*"minist" + 0.023*"member" + 0.018*"state" + 0.017*"servic" + 0.015*"gener" + 0.014*"chief"
2019-02-04 15:37:37,372 : INFO : topic #6 (0.020): 0

2019-02-04 15:37:51,075 : INFO : topic #8 (0.020): 0.026*"law" + 0.023*"court" + 0.018*"state" + 0.016*"act" + 0.012*"right" + 0.012*"case" + 0.010*"report" + 0.010*"polic" + 0.008*"legal" + 0.008*"order"
2019-02-04 15:37:51,076 : INFO : topic #1 (0.020): 0.053*"china" + 0.043*"chines" + 0.025*"hong" + 0.025*"kong" + 0.021*"korea" + 0.018*"korean" + 0.017*"lee" + 0.016*"singapor" + 0.016*"south" + 0.014*"kim"
2019-02-04 15:37:51,077 : INFO : topic #32 (0.020): 0.051*"district" + 0.046*"popul" + 0.043*"villag" + 0.035*"town" + 0.033*"counti" + 0.023*"municip" + 0.022*"ag" + 0.022*"area" + 0.019*"censu" + 0.018*"commun"
2019-02-04 15:37:51,078 : INFO : topic #5 (0.020): 0.038*"album" + 0.028*"song" + 0.027*"releas" + 0.025*"record" + 0.021*"band" + 0.016*"music" + 0.016*"singl" + 0.014*"chart" + 0.013*"track" + 0.010*"vocal"
2019-02-04 15:37:51,083 : INFO : topic diff=0.002997, rho=0.020232
2019-02-04 15:37:51,249 : INFO : PROGRESS: pass 0, at document #4888000/4922894
2019-02-04 15:37:5

2019-02-04 15:38:02,196 : INFO : topic #28 (0.020): 0.037*"build" + 0.031*"hous" + 0.019*"built" + 0.015*"histor" + 0.012*"list" + 0.011*"site" + 0.011*"centuri" + 0.011*"design" + 0.011*"construct" + 0.010*"place"
2019-02-04 15:38:02,197 : INFO : topic #16 (0.020): 0.055*"king" + 0.030*"princ" + 0.020*"duke" + 0.019*"royal" + 0.018*"iii" + 0.018*"queen" + 0.017*"grand" + 0.016*"kingdom" + 0.014*"portugues" + 0.012*"crown"
2019-02-04 15:38:02,203 : INFO : topic diff=0.003133, rho=0.020207
2019-02-04 15:38:04,978 : INFO : -11.399 per-word bound, 2701.2 perplexity estimate based on a held-out corpus of 2000 documents with 546394 words
2019-02-04 15:38:04,979 : INFO : PROGRESS: pass 0, at document #4900000/4922894
2019-02-04 15:38:06,350 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-04 15:38:06,616 : INFO : topic #3 (0.020): 0.031*"presid" + 0.025*"nation" + 0.025*"offic" + 0.023*"govern" + 0.023*"minist" + 0.022*"member" + 0.019*"servic" + 0.018*"

2019-02-04 15:38:15,871 : INFO : topic #12 (0.020): 0.009*"number" + 0.007*"function" + 0.007*"gener" + 0.006*"point" + 0.006*"time" + 0.006*"space" + 0.006*"exampl" + 0.006*"set" + 0.006*"measur" + 0.006*"valu"
2019-02-04 15:38:15,877 : INFO : topic diff=0.003394, rho=0.020182
2019-02-04 15:38:16,029 : INFO : PROGRESS: pass 0, at document #4912000/4922894
2019-02-04 15:38:17,399 : INFO : merging changes from 2000 documents into a model of 4922894 documents
2019-02-04 15:38:17,666 : INFO : topic #49 (0.020): 0.043*"india" + 0.031*"indian" + 0.013*"pakistan" + 0.013*"islam" + 0.012*"arab" + 0.011*"templ" + 0.011*"khan" + 0.011*"africa" + 0.010*"muslim" + 0.010*"tamil"
2019-02-04 15:38:17,667 : INFO : topic #6 (0.020): 0.068*"film" + 0.024*"episod" + 0.021*"seri" + 0.018*"televis" + 0.014*"star" + 0.012*"role" + 0.012*"produc" + 0.010*"direct" + 0.010*"movi" + 0.010*"actor"
2019-02-04 15:38:17,668 : INFO : topic #41 (0.020): 0.044*"citi" + 0.025*"park" + 0.019*"new" + 0.018*"street" + 0.

2019-02-04 15:38:29,671 : INFO : topic diff=0.003322, rho=0.020158
2019-02-04 15:38:31,154 : INFO : -11.497 per-word bound, 2890.3 perplexity estimate based on a held-out corpus of 894 documents with 240967 words
2019-02-04 15:38:31,154 : INFO : PROGRESS: pass 0, at document #4922894/4922894
2019-02-04 15:38:31,914 : INFO : merging changes from 894 documents into a model of 4922894 documents
2019-02-04 15:38:32,180 : INFO : topic #11 (0.020): 0.023*"john" + 0.011*"william" + 0.011*"david" + 0.011*"jame" + 0.010*"robert" + 0.009*"michael" + 0.008*"smith" + 0.008*"georg" + 0.008*"paul" + 0.008*"richard"
2019-02-04 15:38:32,182 : INFO : topic #18 (0.020): 0.011*"time" + 0.007*"later" + 0.006*"said" + 0.006*"dai" + 0.005*"kill" + 0.005*"like" + 0.005*"return" + 0.005*"end" + 0.004*"call" + 0.004*"help"
2019-02-04 15:38:32,183 : INFO : topic #40 (0.020): 0.087*"univers" + 0.023*"scienc" + 0.022*"institut" + 0.021*"colleg" + 0.020*"research" + 0.019*"studi" + 0.014*"professor" + 0.012*"work"

### Load LDA and store metrics

In [17]:
lda = LdaModel.load('lda.model')
row.update(get_tm_metrics(lda, test_corpus))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

2019-02-04 15:38:32,410 : INFO : loading LdaModel object from lda.model
2019-02-04 15:38:32,412 : INFO : loading expElogbeta from lda.model.expElogbeta.npy with mmap=None
2019-02-04 15:38:32,416 : INFO : setting ignored attribute id2word to None
2019-02-04 15:38:32,417 : INFO : setting ignored attribute dispatcher to None
2019-02-04 15:38:32,417 : INFO : setting ignored attribute state to None
2019-02-04 15:38:32,418 : INFO : loaded lda.model
2019-02-04 15:38:32,418 : INFO : loading LdaState object from lda.model.state
2019-02-04 15:38:32,508 : INFO : loaded lda.model.state
/home/anotherbugmaster/gensim/gensim/matutils.py:503: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = np.column_stack(sparse2full(doc, num_terms) for doc in corpus)
2019-02-04 15:38:40,772 : INFO : CorpusAccumulator accumulated stats from 100

### Train Sklearn NMF and store metrics

In [18]:
row = dict()
row['model'] = 'sklearn_nmf'
sklearn_nmf = SklearnNmf(n_components=50, tol=1e-2, random_state=42)
row['train_time'], row['mean_ram'], row['max_ram'], sklearn_nmf = get_train_time_and_ram(
    lambda: sklearn_nmf.fit(scipy.sparse.load_npz('train_csc.npz').T),
    'sklearn_nmf',
)

joblib.dump(sklearn_nmf, 'sklearn_nmf.joblib')

['sklearn_nmf.joblib']

### Load Sklearn NMF and store metrics

In [19]:
sklearn_nmf = joblib.load('sklearn_nmf.joblib')
row.update(get_sklearn_metrics(
    sklearn_nmf, scipy.sparse.load_npz('test_csc.npz').toarray(), dictionary
))
tm_metrics = tm_metrics.append(pd.Series(row), ignore_index=True)

In [20]:
tm_metrics.replace(np.nan, '-', inplace=True)

## Results

In [21]:
tm_metrics.drop('topics', axis=1)

,model,train_time,mean_ram,max_ram,perplexity,coherence,l2_norm
0,gensim_nmf,00:26:09,134 MB,168 MB,3817.239000,-2.9077,1986.291000
1,lda,01:27:46,140 MB,140 MB,4701.976000,-2.5286,2273.643000
2,sklearn_nmf,00:50:30,10753 MB,16407 MB,3943.036299,-,1987.280856


### Insights

Gensim NMF is better than Sklearn NMF in every aspect:

- **2x** faster


- Uses **80x-120x** less memory.

    About **8GB** of RAM comes from the input corpus sparse matrices, which, in contrast to Gensim NMF, can't be passed iteratively. But even if we forget about tremendous corpus size, Sklearn NMF still uses about **2-8 GB** of RAM, which is much larger than that of Gensim NMF and LDA.


- Still achieves better l2 norm and perplexity

Comparing to LDA, Gensim NMF is also better in almost everything:

- **3.5x** faster
- Achieves much better l2 norm and perplexity

Coherence is less than LDA's though.

### Topics

In [22]:
def compare_topics(tm_metrics):
    for _, row in tm_metrics.iterrows():
        print('\n{}:'.format(row.model))
        print("\n".join(str(topic) for topic in row.topics))
        
compare_topics(tm_metrics)


gensim_nmf:
(19, '0.015*"war" + 0.012*"centuri" + 0.011*"king" + 0.007*"french" + 0.006*"battl" + 0.005*"empir" + 0.005*"franc" + 0.005*"british" + 0.004*"histori" + 0.004*"roman"')
(23, '0.055*"music" + 0.029*"ye" + 0.029*"warrenpoint" + 0.026*"station" + 0.020*"radio" + 0.019*"piano" + 0.015*"alt" + 0.011*"orchestra" + 0.009*"compos" + 0.009*"elegan"')
(28, '0.109*"order" + 0.056*"regul" + 0.048*"amend" + 0.028*"road" + 0.022*"traffic" + 0.022*"temporari" + 0.021*"prohibit" + 0.017*"trunk" + 0.014*"junction" + 0.012*"health"')
(44, '0.046*"regiment" + 0.041*"armi" + 0.034*"divis" + 0.028*"battalion" + 0.028*"infantri" + 0.021*"brigad" + 0.018*"corp" + 0.016*"command" + 0.016*"artilleri" + 0.015*"forc"')
(32, '0.077*"district" + 0.047*"school" + 0.025*"educ" + 0.024*"grade" + 0.024*"pennsylvania" + 0.023*"student" + 0.020*"state" + 0.017*"fund" + 0.016*"level" + 0.016*"basic"')

lda:
(35, '0.054*"russian" + 0.039*"soviet" + 0.032*"russia" + 0.027*"polish" + 0.026*"republ" + 0.025*"ph

Seems like all models have successfully learned the topic representation of the corpus.